In [9]:
import numpy as np
import imp
import math
import h5py
import glob, os, sys
import matplotlib.pyplot as plt
import datetime
from datetime import datetime, timedelta
from datetime import date
from optparse import OptionParser
import LORAparameters as LORA
import process_functions
import read_functions as read
import detector
import event


def ProcessEventFinal(outputdir, data_dir, LOFAR_id):

    #generate timestamp
    timestamp=int(float(LOFAR_id))+LORA.event_id_offset
    
    dt_object = datetime.utcfromtimestamp(timestamp)

    year=dt_object.year
    month=dt_object.month
    day=dt_object.day
    hour=dt_object.hour
    minute=dt_object.minute
    sec=dt_object.second

    print('LORAdata-'+str(dt_object.year)+str(dt_object.month).zfill(2)+str(dt_object.day).zfill(2)+'T'+str(dt_object.hour).zfill(2)+str(dt_object.minute).zfill(2)+str(dt_object.second).zfill(2)+'.dat')

    time_code = int(str(dt_object.year)+str(dt_object.month).zfill(2)+str(dt_object.day).zfill(2))

    if time_code <= 20191116:
        detectors, ev = read.load_in_data_V1(LOFAR_id, data_dir)

    else:
        detectors, ev = read.load_in_data_V2(LOFAR_id, data_dir)
    

    #calculate arrival time and direction
    event.do_arrival_time_diff(detectors)
    event.do_arrival_direction(detectors,ev)

    #get density, direction and core location
    event.do_COM_core(detectors,ev)
    event.find_density(detectors,ev)
    event.fit_arrival_direction_trial(detectors,ev)
    event.NKG_fit_trial(detectors,ev)

    #write in output file
    if ev.event_flag==0:# and ev.direction_flag==0:
        print('saving to file')
            
        
        outputfile=open(outputdir+'LORAdata-'+str(dt_object.year)+str(dt_object.month).zfill(2)+str(dt_object.day).zfill(2)+'T'+str(dt_object.hour).zfill(2)+str(dt_object.minute).zfill(2)+str(dt_object.second).zfill(2)+'.dat','w')
        outputfile.write('//UTC_Time(secs)\tnsecs\t\tCore(X)\t\tCore(Y)\t\tElevation\tAzimuth\t\tEnergy(eV)\tCoreE(X)\tCoreE(Y)\tMoliere_rad(m)\t\trM_error\tElevaErr\tAziErr\tEnergyErr(eV)\tNe\t\tNeErr\t\tCorCoef_XY\t\tNe_RefA\t\tNeErr_RefA\t\tEnergy_RefA\t\tEnergyErr_RefA\n')
        outputfile.write('{0}\t{1}\t{2:.2f}\t{3:.2f}\t{4:.2f}\t{5:.2f}\t{6}\t{7:.2f}\t{8:.2f}\t{9:.2f}\t{10:.2f}\t{11:.2f}\t{12}\t{13}\t{14}\t{15}\t{16}\t{17}\t{18}\t{19}\t{20}\n'.format(int(ev.UTC_min), int(ev.nsec_min/10), ev.x_core, ev.y_core, ev.fit_elevation, ev.fit_phi, ev.energy*np.power(10,15), ev.x_core_err, ev.y_core_err, ev.Rm, ev.rM_err, ev.fit_elevation_err, ev.fit_phi_err, ev.energy_err*np.power(10,15), ev.Ne, ev.Ne_err, ev.CorCoef_xy, ev.Ne_RefA, ev.NeErr_RefA, ev.Energy_RefA*np.power(10,15), ev.EnergyErr_RefA*np.power(10,15)))

        outputfile.write('//Detector coordinates w.r.t CS002 LBA center\n')
        outputfile.write('//Det_no.    X_Cord(m)    Y_Cord(m)    Z_Cord(m)    UTC_time    (10*nsecs)  int_ADC_count  gain(ADC/muon)    Particle_Density(/m2), baseline(ADC), rms(ADC), corrected_peak(ADC), corrected_threshold(ADC), avg_mean(ADC), avg_sigma(ADC), trigger?\n')

        for i in np.arange(len(detectors)):
            outputfile.write('{0}\t{1}\t{2}\t{3}\t{4}\t{5}\t{6:.4f}\t{7}\t{8:.4f}\t{9:.2f}\t{10:.2f}\t{11:.2f}\t{12:.2f}\t{13:.2f}\t{14:.2f}\t{15}\n'.format(i+1,detectors[i].x_cord,detectors[i].y_cord,detectors[i].z_cord,int(detectors[i].gps),int(detectors[i].cal_time),detectors[i].trace_int_counts,detectors[i].gain,detectors[i].density,detectors[i].trace_mean,detectors[i].trace_rms,detectors[i].corrected_peak,detectors[i].corrected_threshold,detectors[i].sec_mean,detectors[i].sec_sigma, detectors[i].trig))
            
        outputfile.close() 

        file_name_h5 = ('counts' + 'LORAdata-' +str(dt_object.year) + str(dt_object.month).zfill(2) + str(dt_object.day).zfill(2) +'T' + str(dt_object.hour).zfill(2) + str(dt_object.minute).zfill(2) + str(dt_object.second).zfill(2) + '.h5')

        #output counts data to h5 file
        num_detectors = len(detectors)
        num_counts = len(detectors[0].counts)
        counts_array = np.zeros((num_counts, num_detectors)) 

        for i in range(num_detectors):
            counts_array[:, i] = detectors[i].counts

        with h5py.File(outputdir + file_name_h5, 'w') as hdf:
            hdf.create_dataset('counts', data=counts_array, dtype='int')

        

In [6]:
imp.reload(read)
#20190903T150207
#20211013T020156


<module 'read_functions' from '/vol/astro7/lofar/lvdongen/LORA_V2_final_version/read_functions.py'>

In [8]:
filestamp= '20211013T020156'

yearE=int(filestamp[0:4])
monthE=int(filestamp[4:6])
dayE=int(filestamp[6:8])
hourE=int(filestamp[9:11])
minuteE=int(filestamp[11:13])
secE=int(filestamp[13:15])
dt = datetime(year=yearE, month=monthE, day=dayE,hour=hourE,minute=minuteE,second=secE)
timestamp = int((dt - datetime(1970, 1, 1)).total_seconds())



out = '/vol/astro7/lofar/lvdongen/OutputV2root/'
src = '/vol/astro5/lofar/astro3/vhecr/lora_triggered/LORAraw/'
l_id = timestamp-LORA.event_id_offset

ProcessEventFinal(out, src, l_id)

LORAdata-20211013T020156.dat
no_match 1634090516
_____________________
no_match:    1634090516   274489170
no log file found, doing this the hard way
no standard log file avaliable, trying to look at .root files
did we find the index?   -1
did we find the index?   -1
did we find the index?   510
returning file tag
---> 20211013_0030
can't find log file
reading root file: /vol/astro5/lofar/astro3/vhecr/lora_triggered/LORAraw/20211013_0030.root
Finding threshold for detector 1: 41.0
Finding threshold for detector 2: 41.0
Finding threshold for detector 3: 41.0
Finding threshold for detector 4: 41.0
Finding threshold for detector 5: 29.0
Finding threshold for detector 6: 29.0
Finding threshold for detector 7: 34.0
Finding threshold for detector 8: 34.0
Finding threshold for detector 9: 34.0
Finding threshold for detector 10: 34.0
Finding threshold for detector 11: 34.0
Finding threshold for detector 12: 34.0
Finding threshold for detector 13: 48.0
Finding threshold for detector 14: 47.0
Fi

In [ ]:
file_list = glob.glob('/vol/astro5/lofar/astro3/vhecr/lora_triggered/LORA/LORAdata-202110*.dat')

print(len(file_list))

for i in np.arange(len(file_list)):

    #convert filestamp to lora id
    filestamp=file_list[i][-19:-4]
    print(filestamp)
    yearE=int(filestamp[0:4])
    monthE=int(filestamp[4:6])
    dayE=int(filestamp[6:8])
    hourE=int(filestamp[9:11])
    minuteE=int(filestamp[11:13])
    secE=int(filestamp[13:15])
    dt = datetime(year=yearE, month=monthE, day=dayE,hour=hourE,minute=minuteE,second=secE)
    timestamp = int((dt - datetime(1970, 1, 1)).total_seconds())
    
    
    #set output, data directory and lofar id
    out = '/vol/astro7/lofar/lvdongen/OutputV2root/'
    src = '/vol/astro5/lofar/astro3/vhecr/lora_triggered/LORAraw/'
    l_id = timestamp-LORA.event_id_offset
    try:
        ProcessEventV2(out, src, l_id)
    
    except OSError as error :
        print('root file not found, moving on') 

445
20211015T184823
LORAdata-20211015T184823.dat
no_match 1634323703
_____________________
no_match:    1634323703   570732317
no log file found, doing this the hard way
no standard log file avaliable, trying to look at .root files
['/vol/astro5/lofar/astro3/vhecr/lora_triggered/LORAraw/20211013_0030.root', '/vol/astro5/lofar/astro3/vhecr/lora_triggered/LORAraw/20211014_0030.root', '/vol/astro5/lofar/astro3/vhecr/lora_triggered/LORAraw/20211015_0030.root']
did we find the index?   -1
did we find the index?   -1
did we find the index?   -1
---> no_match
can't find log file
root file not found, moving on
20211015T215454
LORAdata-20211015T215454.dat
no_match 1634334894
_____________________
no_match:    1634334894   570732317
no log file found, doing this the hard way
no standard log file avaliable, trying to look at .root files
['/vol/astro5/lofar/astro3/vhecr/lora_triggered/LORAraw/20211013_0030.root', '/vol/astro5/lofar/astro3/vhecr/lora_triggered/LORAraw/20211014_0030.root', '/vol/ast

KeyboardInterrupt: 

Error in <TFile::TFile>: file /vol/astro5/lofar/astro3/vhecr/lora_triggered/LORAraw/no_match.root does not exist
Error in <TFile::TFile>: file /vol/astro5/lofar/astro3/vhecr/lora_triggered/LORAraw/no_match.root does not exist
Error in <TFile::TFile>: file /vol/astro5/lofar/astro3/vhecr/lora_triggered/LORAraw/no_match.root does not exist
Error in <TFile::TFile>: file /vol/astro5/lofar/astro3/vhecr/lora_triggered/LORAraw/no_match.root does not exist
Error in <TFile::TFile>: file /vol/astro5/lofar/astro3/vhecr/lora_triggered/LORAraw/no_match.root does not exist
Error in <TFile::TFile>: file /vol/astro5/lofar/astro3/vhecr/lora_triggered/LORAraw/no_match.root does not exist
Error in <TFile::TFile>: file /vol/astro5/lofar/astro3/vhecr/lora_triggered/LORAraw/no_match.root does not exist
Error in <TFile::TFile>: file /vol/astro5/lofar/astro3/vhecr/lora_triggered/LORAraw/no_match.root does not exist
Error in <TFile::TFile>: file /vol/astro5/lofar/astro3/vhecr/lora_triggered/LORAraw/no_match.roo